In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, metrics, tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

In [2]:
df = pd.read_csv('df.csv')
df.head(5)

,Unnamed: 0,age,marital,education,default,balance,housing,loan,day,campaign,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0,58,1.0,3.0,0,2143,1,0,5,1,...,0,0,1,0,0,0,0,0,0,1
1,1,44,0.0,2.0,0,29,1,0,5,1,...,0,0,1,0,0,0,0,0,0,1
2,2,33,1.0,2.0,0,2,1,1,5,1,...,0,0,1,0,0,0,0,0,0,1
3,3,47,1.0,0.0,0,1506,1,0,5,1,...,0,0,1,0,0,0,0,0,0,1
4,4,33,0.0,0.0,0,1,0,0,5,1,...,0,0,1,0,0,0,0,0,0,1


In [3]:
X = df.drop('y', axis=1)
y = df['y']

y = pd.DataFrame(y).reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1))

C:\Users\User\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
# Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2), 
    Dense(64, activation='relu'),
    Dense(y_train_encoded.shape[1], activation='softmax') 
])
 
model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy']) 

C:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,018 (54.76 KB)

 Trainable params: 14,018 (54.76 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.layers

[<Dense name=dense, built=True>,
 <Dropout name=dropout, built=True>,
 <Dense name=dense_1, built=True>,
 <Dense name=dense_2, built=True>]

In [9]:
# Fitting
history = model.fit(X_train_scaled, y_train_encoded,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/20
905/905 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8742 - loss: 0.3388 - val_accuracy: 0.8920 - val_loss: 0.2933
Epoch 2/20
905/905 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8943 - loss: 0.2874 - val_accuracy: 0.8918 - val_loss: 0.2953
Epoch 3/20
905/905 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8985 - loss: 0.2725 - val_accuracy: 0.8878 - val_loss: 0.2899
Epoch 4/20
905/905 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8981 - loss: 0.2713 - val_accuracy: 0.8918 - val_loss: 0.2888
Epoch 5/20
905/905 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8968 - loss: 0.2755 - val_accuracy: 0.8936 - val_loss: 0.2891
Epoch 6/20
905/905 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8990 - loss: 0.2734 - val_accuracy: 0.8926 - val_loss: 0.2887
Epoch 7/20
905/905 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9000 - loss: 0.2721 - val_accuracy: 0.8933 - val_loss: 0.2877
Epoch 8/20
905/905 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8966 - loss: 0.2727 - val_accuracy: 0.

In [10]:
# Evaluate
loss, accuracy = model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
 
# Making predictions
predictions = model.predict(X_test_scaled)
predictions_binary = np.argmax(predictions, axis=1)  
y_test_binary = np.argmax(y_test_encoded, axis=1)
 
# Calculate Precision and Recall
precision = precision_score(y_test_binary, predictions_binary, average='macro')  
recall = recall_score(y_test_binary, predictions_binary, average='macro') 
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8919 - loss: 0.2986
Test Loss: 0.2931925654411316, Test Accuracy: 0.891960620880127
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.891960620880127, Precision: 0.7723908329599021, Recall: 0.609581238577201


In [11]:
from keras import models  
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Activation  
from keras import layers
from keras_visualizer import visualizer
from keras.utils import plot_model
import pydotplus

In [12]:
model1 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(43,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

plot_model(model1, to_file='model1.png', show_shapes=True, show_layer_names=True, expand_nested=True, dpi=300)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


C:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
